In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio # pip install imageio
import 

In [84]:
letterDict={
  "a": 0,"b": 1,"c": 2,"d": 3,"e": 4,"f": 5,"g": 6,"h": 7,"i": 8,
    "j": 9,"k": 10,"l": 11,"m": 12,"n": 13,"o": 14,"p": 15,"q": 16,"r": 17,
    "s": 18,"t": 19,"u": 20,"v": 21,"w": 22,"x": 23,"y": 24,"z": 25
}

def readPictures(df_complete, directory,letter_value):
    im = imageio.imread(directory)
    im.shape
    im=np.reshape(im,400)
    df = pd.DataFrame(data=im).T
    y = np.repeat(0,400)
    df['Letter'] = letter_value
    
    return df_complete.append(df)


In [62]:

y = np.repeat(0,400)


In [101]:
import os
all_imgs = pd.DataFrame()

def read_all_images(upper_directory,all_imgs):
    for subdir, dirs, files in os.walk(upper_directory):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg"):
                x = filepath.split("/")
                letter = (x[-1][0])
                letter_val = letterDict.get(letter)
                all_imgs = readPictures(all_imgs,filepath,letter_val)
    return all_imgs

            
alldata = read_all_images("./dataset3",all_imgs)
alldata

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,Letter
0,193,175,177,191,177,188,194,177,197,157,...,175,191,175,183,188,225,185,167,193,0
0,37,34,37,46,50,45,40,39,39,38,...,34,34,35,36,37,36,36,37,38,0
0,25,33,40,42,40,38,36,35,42,46,...,33,38,38,32,31,51,43,41,41,0
0,27,34,36,30,31,35,32,23,20,12,...,24,23,23,27,40,8,27,33,20,0
0,186,192,175,189,202,182,181,188,191,193,...,194,196,185,182,196,193,190,191,194,0
0,34,37,7,25,14,19,22,26,34,13,...,14,15,19,20,17,8,30,21,15,0
0,40,5,5,0,34,8,16,38,22,40,...,35,12,0,57,31,29,6,47,59,0
0,232,251,223,238,237,221,235,232,231,216,...,20,37,35,42,22,56,37,41,13,0
0,232,228,228,229,230,234,212,138,45,45,...,232,235,199,69,26,34,54,49,96,0
0,223,229,220,216,226,222,217,231,238,225,...,229,234,228,222,223,227,227,227,227,0


In [103]:
alldata = alldata.sample(frac=1)
alldata


,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,Letter
0,6,5,9,0,23,27,9,19,18,26,...,40,45,46,45,43,45,46,47,48,11
0,236,222,230,227,253,229,237,255,223,237,...,222,228,228,215,229,233,233,234,234,24
0,39,21,30,30,24,44,18,35,39,29,...,30,38,35,33,38,26,28,29,30,2
0,245,237,233,241,238,238,246,236,241,246,...,232,240,237,231,237,233,238,240,238,13
0,38,21,26,29,35,14,13,12,0,17,...,29,26,25,31,32,23,29,31,26,23
0,166,171,175,176,175,175,178,182,164,158,...,44,44,45,47,47,56,51,45,45,8
0,18,27,20,34,35,42,20,43,32,31,...,31,29,28,26,23,26,25,24,24,3
0,61,90,29,58,46,39,33,51,49,44,...,107,70,70,107,29,44,50,53,49,18
0,0,0,12,0,6,11,5,6,9,2,...,31,6,0,7,0,4,1,2,8,3
0,120,142,154,107,149,98,72,97,55,130,...,98,49,49,69,144,152,140,8,108,9


In [ ]:
## *** Data augmentation *** ##

